<a href="https://colab.research.google.com/github/OziomaEunice/Sentiment_GPT/blob/develop/Na%C3%AFveBayes_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Naïve Bayes for Sentiment Analysis**

## **Training**

In [ ]:
#

## **Testing**

In [ ]:
#